# FooDB.ca Data Analysis

<a id = 'top'></a>

# Contents

- [Libraries and Functions]
- [Exploratory Data Analysis](#eda)
- [Dropping columns from the db](#droppingcolumns)
-- Food Taxonomies table
--- [Test query of morels](#test_query_ft)
- [Dropping whole tables](#dropping_tables)

# Libraries and Functions

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
#For the ORM:
import sqlalchemy
from sqlalchemy import create_engine #To connect to the database
from sqlalchemy.orm import Session, sessionmaker #To interact with the database as object
from sqlalchemy import inspect #To look at tables and column names

In [ ]:
#import plotly.plotly as py
#import plotly.figure_factory as ff

In [ ]:
#needs plotly password
#df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")

#table = ff.create_table(df)
#py.iplot(table, filename='jupyter-table1')

In [2]:
#Instantiate a session of the ORM
#Connect to the sqlite database file
engine = create_engine('sqlite:///foodb.db', echo = True)
#Bind a session factory to the engine
Session = sessionmaker(bind=engine)
#Instantiate a session
session = Session()

In [3]:
#An object to get a first look at the database
inspector = inspect(engine)

2019-03-15 12:18:58,658 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-03-15 12:18:58,668 INFO sqlalchemy.engine.base.Engine ()
2019-03-15 12:18:58,668 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-03-15 12:18:58,668 INFO sqlalchemy.engine.base.Engine ()


In [4]:
#Make a dict of list of names of all the columns in all the tables
#for easy reference

#Get list of all table names
table_names = inspector.get_table_names()

tables_columns = dict()
#For each table name
for table in table_names:
    column_names = []
    #for each column in the table, get just the name
    for column in inspector.get_columns(table):
        column_names.append(column['name'])
    tables_columns[table] = column_names

2019-03-15 12:19:01,643 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-03-15 12:19:01,644 INFO sqlalchemy.engine.base.Engine ()
2019-03-15 12:19:01,679 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("compound_alternate_parents")
2019-03-15 12:19:01,680 INFO sqlalchemy.engine.base.Engine ()
2019-03-15 12:19:01,682 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("compound_external_descriptors")
2019-03-15 12:19:01,683 INFO sqlalchemy.engine.base.Engine ()
2019-03-15 12:19:01,685 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("compound_substituents")
2019-03-15 12:19:01,685 INFO sqlalchemy.engine.base.Engine ()
2019-03-15 12:19:01,688 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("compound_synonyms")
2019-03-15 12:19:01,689 INFO sqlalchemy.engine.base.Engine ()
2019-03-15 12:19:01,690 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("compounds")
2019-03-15 12:19:01,691 INFO sqlalchemy.engine.base.Engine (

<a id = 'eda'></a>

[(Back to top)](#top)

# Exploratory Data Analysis
## Foods table

In [10]:
foods = pd.read_sql_query('''SELECT f.* 
                            FROM foods f
                            ;''', engine)

2019-03-15 12:25:32,098 INFO sqlalchemy.engine.base.Engine SELECT f.* 
                            FROM foods f
                            ;
2019-03-15 12:25:32,098 INFO sqlalchemy.engine.base.Engine ()


In [27]:
foods.head()

,id,name,name_scientific,description,itis_id,wikipedia_id,wikipedia_id_img,picture_content_type,picture_file_size,picture_updated_at,...,food_subgroup,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb
0,1,Angelica,Angelica keiskei,Angelica is a genus of about 60 species of tal...,,Angelica,1.jpg,image/jpeg,111325,2012-04-20 09:29:57 UTC,...,Herbs,Type 1,2011-02-09 00:37:14 UTC,2017-06-27 17:13:48 UTC,,2,FALSE,specific,357850,TRUE
1,2,Savoy cabbage,Brassica oleracea var. sabauda,Savoy cabbage (Brassica oleracea convar. capit...,,Savoy cabbage,2.jpg,image/jpeg,155178,2012-04-20 09:39:54 UTC,...,Cabbages,Type 1,2011-02-09 00:37:15 UTC,2017-03-20 21:26:22 UTC,,,FALSE,specific,1216010,TRUE
2,3,Silver linden,Tilia argentea,Tilia tomentosa (Silver Lime in the UK and Sil...,,Tilia tomentosa,3.jpg,image/jpeg,56367,2012-04-20 09:41:25 UTC,...,Herbs,Type 1,2011-02-09 00:37:15 UTC,2015-10-02 21:24:39 UTC,,,FALSE,specific,,TRUE
3,4,Kiwi,Actinidia chinensis,"The kiwifruit, often shortened to kiwi in many...",506775,Kiwifruit,4.jpg,image/jpeg,110661,2012-04-20 09:32:21 UTC,...,Tropical fruits,Type 1,2011-02-09 00:37:15 UTC,2017-03-20 21:26:22 UTC,,,FALSE,specific,3625,TRUE
4,5,Allium (Onion),Allium,Allium haematochiton is a species of wild onio...,42634,Allium haematochiton,5.jpg,image/jpeg,341911,2012-04-20 09:37:44 UTC,...,Onion-family vegetables,Type 1,2011-02-09 00:37:15 UTC,2017-03-20 21:26:22 UTC,,,FALSE,specific,4678,TRUE


In [69]:
foods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 22 columns):
id                      907 non-null object
name                    907 non-null object
name_scientific         907 non-null object
description             907 non-null object
itis_id                 907 non-null object
wikipedia_id            907 non-null object
wikipedia_id_img        907 non-null object
picture_content_type    907 non-null object
picture_file_size       907 non-null object
picture_updated_at      907 non-null object
legacy_id               907 non-null object
food_group              907 non-null object
food_subgroup           907 non-null object
food_type               907 non-null object
created_at              907 non-null object
updated_at              907 non-null object
creator_id              907 non-null object
updater_id              907 non-null object
export_to_afcdb         907 non-null object
category                907 non-null object
ncbi_taxonomy_i

Columns to drop:

picture_updated_at	

created_at

updated_at

creator_id

updater_id

export_to_afcdb

export_to_foodb

In [24]:
foods.shape

(907, 22)

In [25]:
foods['name'].value_counts().sum(), foods['name'].value_counts().sum(), foods['id'].value_counts().sum()

(907, 907, 907)

### There are 907 foods in the database.

In [9]:
tables_columns['foods']

['id',
 'name',
 'name_scientific',
 'description',
 'itis_id',
 'wikipedia_id',
 'wikipedia_id_img',
 'picture_content_type',
 'picture_file_size',
 'picture_updated_at',
 'legacy_id',
 'food_group',
 'food_subgroup',
 'food_type',
 'created_at',
 'updated_at',
 'creator_id',
 'updater_id',
 'export_to_afcdb',
 'category',
 'ncbi_taxonomy_id',
 'export_to_foodb']

In [28]:
#alterante for food subgroup valuecounts
#pd.read_sql_query('''SELECT f.food_subgroup, 
#                        COUNT(f.name) foods
 #                       FROM foods f
 #                       GROUP BY f.food_subgroup
  #                      ORDER BY foods DESC
  #                  ;''', engine)

### Investigating Type 1 and Type 2

In [29]:
pd.read_sql_query('''SELECT f.food_subgroup, 
                        COUNT(f.food_subgroup) subgroup
                        FROM foods f
                        WHERE f.food_type == 'Type 1'
                        GROUP BY f.food_subgroup
                    ;''', engine)

2019-03-12 19:13:32,229 INFO sqlalchemy.engine.base.Engine SELECT f.food_subgroup, 
                        COUNT(f.food_subgroup) subgroup
                        FROM foods f
                        WHERE f.food_type == 'Type 1'
                        GROUP BY f.food_subgroup
                    ;
2019-03-12 19:13:32,230 INFO sqlalchemy.engine.base.Engine ()


,food_subgroup,subgroup
0,Amphibians,1
1,Beans,14
2,Berries,55
3,Bovines,4
4,Cabbages,20
5,Caprae,1
6,Cereals,18
7,Cetaceans,3
8,Citrus,8
9,Cocoa,1


In [30]:
pd.read_sql_query('''SELECT f.food_subgroup, 
                        COUNT(f.food_subgroup) subgroup
                        FROM foods f
                        WHERE f.food_type == 'Type 2'
                        GROUP BY f.food_subgroup
                    ;''', engine)

2019-03-12 19:14:06,827 INFO sqlalchemy.engine.base.Engine SELECT f.food_subgroup, 
                        COUNT(f.food_subgroup) subgroup
                        FROM foods f
                        WHERE f.food_type == 'Type 2'
                        GROUP BY f.food_subgroup
                    ;
2019-03-12 19:14:06,830 INFO sqlalchemy.engine.base.Engine ()


,food_subgroup,subgroup
0,Alcoholic beverages,2
1,American cuisine,3
2,Animal fats,4
3,Asian cuisine,2
4,Baby foods,1
5,Baking goods,13
6,Berber cuisine,1
7,Bread products,1
8,Candies,8
9,Cereal products,8


### Type 1 looks like unprocessed or unprepared foods. Type 2 foods are processed or prepared.

In [51]:
#foods[foods['food_subgroup'] == 'Marsupials']


In [ ]:
#to drop: id 694 (subgroup Unclassified)
#id 685 (subgroup waters)

### Investigating NCBI Taxonomy ID

In [60]:
#Says no NA values but I see some in the head
foods['ncbi_taxonomy_id'].isna().sum()

0

In [72]:
foods['ncbi_taxonomy_id'].value_counts()b

           276
4072         6
3885         3
13427        3
135518       2
41679        2
4682         2
183260       2
190544       2
4679         2
3661         2
138011       2
37656        2
3714         2
16718        1
357850       1
34256        1
8113         1
66014        1
888065       1
9031         1
59166        1
37796        1
6565         1
9721         1
7797         1
167592       1
109171       1
2763         1
225387       1
          ... 
49390        1
33637        1
195615       1
397755       1
36066        1
8112         1
106975       1
36181        1
42229        1
9103         1
4071         1
334483       1
4558         1
32219        1
6763         1
29780        1
1042646      1
4039         1
119950       1
7935         1
8017         1
3891         1
13493        1
30850        1
485725       1
103480       1
24663        1
403101       1
4681         1
66656        1
Name: ncbi_taxonomy_id, Length: 613, dtype: int64

In [57]:
foods[foods['ncbi_taxonomy_id'] == '4072']

,id,name,name_scientific,description,itis_id,wikipedia_id,wikipedia_id_img,picture_content_type,picture_file_size,picture_updated_at,...,food_subgroup,food_type,created_at,updated_at,creator_id,updater_id,export_to_afcdb,category,ncbi_taxonomy_id,export_to_foodb
39,40,Pepper (C. annuum),Capsicum annuum,<i>Capsicum annuum</i> is a domesticated speci...,30492,Capsicum_annuum,40.jpg,image/jpeg,45222,2012-04-20 09:37:27 UTC,...,Fruit vegetables,Type 1,2011-02-09 00:37:17 UTC,2017-03-20 21:26:23 UTC,,2,FALSE,specific,4072,TRUE
891,909,Green bell pepper,Capsicum annuum,Green bell peppers are members of the domestic...,30492,Capsicum_annuum,428px-Green-Bell-Pepper.jpg,image/jpeg,26784,2015-02-26 18:37:27 UTC,...,Fruit vegetables,Type 1,2015-02-26 18:37:27 UTC,2017-03-20 21:26:33 UTC,,,TRUE,specific,4072,TRUE
892,910,Yellow bell pepper,Capsicum annuum,Yellow bell peppers are members of the domesti...,30492,Capsicum_annuum,bell_pepper_yellow_2.png,image/png,25165,2015-02-26 18:42:45 UTC,...,Fruit vegetables,Type 1,2015-02-26 18:42:45 UTC,2017-03-20 21:26:34 UTC,,,TRUE,specific,4072,TRUE
893,911,Orange bell pepper,Capsicum annuum,Orange bell peppers are members of the domesti...,30492,Capsicum_annuum,orange-bell-pepper-02.jpg,image/jpeg,6881,2015-02-26 18:44:18 UTC,...,Fruit vegetables,Type 1,2015-02-26 18:44:18 UTC,2017-03-20 21:26:34 UTC,,,TRUE,specific,4072,TRUE
894,912,Red bell pepper,Capsicum annuum,Red bell peppers are members of the domesticat...,30492,Capsicum_annuum,red-bell-pepper.jpg,image/jpeg,76376,2015-02-26 18:47:15 UTC,...,Fruit vegetables,Type 1,2015-02-26 18:47:16 UTC,2017-03-20 21:26:34 UTC,,,TRUE,specific,4072,TRUE
895,913,Italian sweet red pepper,Capsicum annuum,The Italian sweet pepper is a variety of the s...,,Italian sweet pepper,939px-Italian_sweet_peppers.jpg,image/jpeg,124244,2015-02-26 18:54:22 UTC,...,Fruit vegetables,Type 1,2015-02-26 18:54:23 UTC,2017-03-20 21:26:34 UTC,,,TRUE,specific,4072,TRUE


### Some ncbis are repeated but all foods seem to have one.

### Investigating Category

In [74]:
foods['category'].value_counts()

specific    881
generic      26
Name: category, dtype: int64

In [100]:
generics = list(foods['name'][foods['category'] == 'generic'])
subgroups = list(set(foods['food_subgroup']))

In [106]:
len(generics)

26

In [103]:
generic_subgroups = []

for sub in subgroups:
    if any(generic in sub for generic in generics):
        generic_subgroups.append(sub)

In [109]:
len(set(generic_subgroups))

27

### The "generic" category is just 26 of the subgroups with a bit more information about each.

# Exploratory data analysis
## Enzymes table

In [23]:
enzymes = pd.read_sql_query('''SELECT e.* 
                            FROM enzymes e
                            ;''', engine)

2019-03-15 12:48:57,422 INFO sqlalchemy.engine.base.Engine SELECT e.* 
                            FROM enzymes e
                            ;


INFO:sqlalchemy.engine.base.Engine:SELECT e.* 
                            FROM enzymes e
                            ;


2019-03-15 12:48:57,422 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [119]:
enzymes.shape

(1744, 32)

In [125]:
enzymes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 32 columns):
id                       1744 non-null object
name                     1744 non-null object
gene_name                1744 non-null object
description              1744 non-null object
go_classification        1744 non-null object
general_function         1744 non-null object
specific_function        1744 non-null object
pathway                  1744 non-null object
reaction                 1744 non-null object
cellular_location        1744 non-null object
signals                  1744 non-null object
transmembrane_regions    1744 non-null object
molecular_weight         1744 non-null object
theoretical_pi           1744 non-null object
locus                    1744 non-null object
chromosome               1744 non-null object
uniprot_name             1744 non-null object
uniprot_id               1744 non-null object
pdb_id                   1744 non-null object
genbank_protein_id 

In [137]:
enzymes['name'].value_counts().sum(), enzymes['gene_name'].value_counts().sum()

(1744, 1744)

### 1744 unique names and gene names. Linked to compounds.

### How many null values?

In [112]:
enzymes.head(3)

,id,name,gene_name,description,go_classification,general_function,specific_function,pathway,reaction,cellular_location,...,genatlas_id,hgnc_id,hprd_id,organism,general_citations,comments,creator_id,updater_id,created_at,updated_at
0,1,UDP-glucuronosyltransferase 2B28,UGT2B28,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2015-02-23 22:40:56,2015-02-23 22:40:56
1,2,Estrogen receptor beta,ESR2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2015-02-23 22:40:57,2015-02-23 22:40:57
2,3,UDP-glucuronosyltransferase 2B4,UGT2B4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,2015-02-23 22:40:57,2015-02-23 22:40:57


In [117]:
null_enzyme_descriptions = enzymes['description'] == 'NULL'

In [118]:
null_enzyme_descriptions.sum()

1744

In [121]:
enzymes[enzymes['description'] == 'NULL'].shape

(1744, 32)

In [122]:
enzymes[enzymes['go_classification'] == 'NULL'].shape

(1744, 32)

In [123]:
enzymes[enzymes['general_function'] == 'NULL'].shape

(1744, 32)

In [124]:
enzymes[enzymes['specific_function'] == 'NULL'].shape

(1744, 32)

In [131]:
for column in enzymes.columns:
    print(column, enzymes[enzymes[column] == 'NULL'].shape[0])

id 0
name 0
gene_name 0
description 1744
go_classification 1744
general_function 1744
specific_function 1744
pathway 1744
reaction 1744
cellular_location 1744
signals 1744
transmembrane_regions 1744
molecular_weight 1744
theoretical_pi 1744
locus 1744
chromosome 1744
uniprot_name 1744
uniprot_id 0
pdb_id 1744
genbank_protein_id 1744
genbank_gene_id 1744
genecard_id 1744
genatlas_id 1744
hgnc_id 1744
hprd_id 1744
organism 1744
general_citations 1744
comments 1744
creator_id 1744
updater_id 1744
created_at 0
updated_at 0


### Drop all columns except id, name, and gene_name.

In [136]:
enzymes['gene_name'].value_counts().sum()

1744

# Exploratory data analysis
## flavors table

In [139]:
flavors = pd.read_sql_query('''SELECT fl.* 
                            FROM flavors fl
                            ;''', engine)

2019-03-12 20:53:19,169 INFO sqlalchemy.engine.base.Engine SELECT fl.* 
                            FROM flavors fl
                            ;
2019-03-12 20:53:19,170 INFO sqlalchemy.engine.base.Engine ()


In [146]:
flavors.head()

,id,name,flavor_group,category,created_at,updated_at,creator_id,updater_id
0,1,celery,vegetable,odor,2011-10-02 06:10:04,2011-10-02 06:10:04,NULL,NULL
1,2,corn,vegetable,odor,2011-10-02 06:10:04,2011-10-02 06:10:04,NULL,NULL
2,3,cucumber,vegetable,odor,2011-10-02 06:10:04,2011-10-02 06:10:04,NULL,NULL
3,4,horseradish,vegetable,odor,2011-10-02 06:10:05,2011-10-02 06:10:05,NULL,NULL
4,5,vegetable,vegetable,odor,2011-10-02 06:10:05,2011-10-02 06:10:05,NULL,NULL


In [150]:
flavors['name'].value_counts()

citral                         1
fungal                         1
dill                           1
clean                          1
paint                          1
maple syrup                    1
coriander                      1
prune                          1
heliotropin                    1
flower shop                    1
laundered cloths               1
cardamom                       1
green leaf                     1
paper                          1
burnt almonds                  1
laundry                        1
fecal                          1
waxy                           1
bark                           1
peppery                        1
cayloxol                       1
benzyl propionate              1
lemongrass                     1
linseed                        1
syrup                          1
cereal                         1
apple skin                     1
mild cooling                   1
pistachio                      1
freshly bruised sage leaves    1
          

In [151]:
flavors['flavor_group'].value_counts()

NULL             743
fruity            24
floral            14
balsamic          10
vegetable          8
fatty              7
wine_like          5
herbaceous         5
citrus             5
nutty              5
woody              4
chemical           3
meaty              3
green              3
earthy             3
spicy              2
musky              1
tobacco            1
smoky              1
minty              1
medicinal          1
camphoraceous      1
mossy              1
fishy              1
pepper             1
seedy              1
animal             1
soapy              1
Name: flavor_group, dtype: int64

In [153]:
flavors['category'].value_counts()

odor    856
Name: category, dtype: int64

In [156]:
table_names

['compound_alternate_parents',
 'compound_external_descriptors',
 'compound_substituents',
 'compound_synonyms',
 'compounds',
 'compounds_enzymes',
 'compounds_flavors',
 'compounds_health_effects',
 'compounds_pathways',
 'enzymes',
 'flavors',
 'food_taxonomies',
 'foodcomex_compound_providers',
 'foodcomex_compounds',
 'foods',
 'health_effects',
 'nutrients',
 'pathways',
 'references']

### Drop all columns except id, name, and flavor_group
### Will have to see later how compounds are connected to these descriptions of flavor. Disappointed that there are 856 unique descriptors ('name') and very few useful flavor_group designations.

# Exploratory Data Analysis
## Health effects table

In [157]:
health_effects = pd.read_sql_query('''SELECT he.* 
                            FROM health_effects he
                            ;''', engine)

2019-03-12 21:02:44,726 INFO sqlalchemy.engine.base.Engine SELECT he.* 
                            FROM health_effects he
                            ;
2019-03-12 21:02:44,728 INFO sqlalchemy.engine.base.Engine ()


In [158]:
health_effects.shape

(1435, 10)

In [159]:
health_effects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1435 entries, 0 to 1434
Data columns (total 10 columns):
id                  1435 non-null object
name                1435 non-null object
description         1435 non-null object
chebi_name          1435 non-null object
chebi_id            1435 non-null object
created_at          1435 non-null object
updated_at          1435 non-null object
creator_id          1435 non-null object
updater_id          1435 non-null object
chebi_definition    1435 non-null object
dtypes: object(10)
memory usage: 112.2+ KB


In [160]:
health_effects.head()

,id,name,description,chebi_name,chebi_id,created_at,updated_at,creator_id,updater_id,chebi_definition
0,1,(+)-inotropic,An agent that alters the force or energy of mu...,NULL,NULL,2012-04-10 10:05:13,2015-11-11 02:32:19,NULL,NULL,NULL
1,2,(-)-chronotropic,An agent that may change theÂ heartÂ rate by a...,NULL,NULL,2012-04-10 10:05:13,2015-11-11 02:32:19,NULL,NULL,NULL
2,3,(-)-inotropic,An agent that alters the force or energy of mu...,NULL,NULL,2012-04-10 10:05:13,2015-11-11 02:32:19,NULL,NULL,NULL
3,4,11beta-hydroxysteroid-dehydrogenase inhibitor,NULL,enzyme inhibitor,23924,2012-04-10 10:05:13,2015-10-27 17:47:35,NULL,NULL,A compound or agent that combines with an enzy...
4,5,12-lipoxygenase inhibitor,NULL,enzyme inhibitor,23924,2012-04-10 10:05:13,2015-10-27 17:47:36,NULL,NULL,A compound or agent that combines with an enzy...


In [161]:
for column in health_effects.columns:
    print(column, health_effects[health_effects[column] != 'NULL'].shape[0])

id 1435
name 1435
description 619
chebi_name 710
chebi_id 710
created_at 1435
updated_at 1435
creator_id 0
updater_id 0
chebi_definition 698


In [162]:
#Are the rows with a non-null chebi_id the same as the rows with a non-null chebi_name?
chebi_ids = health_effects[health_effects['chebi_id'] != 'NULL']

In [165]:
chebi_name = health_effects[health_effects['chebi_name'] != 'NULL']

In [167]:
trues = chebi_ids == chebi_name

### Drop created_at, updated_at, creator_id, updater_id

# Exploratory Data Analysis
## Nutrients table

In [55]:
nutrients = pd.read_sql_query('''SELECT n.* 
                            FROM nutrients n
                            ;''', engine)

2019-03-15 13:29:47,268 INFO sqlalchemy.engine.base.Engine SELECT n.* 
                            FROM nutrients n
                            ;


INFO:sqlalchemy.engine.base.Engine:SELECT n.* 
                            FROM nutrients n
                            ;


2019-03-15 13:29:47,268 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [67]:
nutrients.head()

,id,legacy_id,type,public_id,name,export,state,annotation_quality,description,wikipedia_id,...,eafus_id,dfc_name,compound_source,metabolism,synthesis_citations,general_citations,creator_id,updater_id,created_at,updated_at
0,1,10930,Nutrient,FDBN00001,Fat,0,NULL,low,NULL,NULL,...,NULL,NULL,DUKE,NULL,NULL,NULL,NULL,NULL,2014-11-05 13:42:10,2014-11-05 13:42:10
1,2,10946,Nutrient,FDBN00002,Proteins,0,NULL,low,NULL,NULL,...,NULL,NULL,DUKE,NULL,NULL,NULL,NULL,NULL,2014-11-05 13:42:15,2014-11-05 13:42:15
2,3,16037,Nutrient,FDBN00003,Carbohydrate,0,NULL,low,Carbohydrates (or saccharides) are organic com...,Carbohydrate,...,NULL,NULL,DUKE,NULL,NULL,NULL,NULL,NULL,2014-11-05 13:44:06,2014-11-05 13:44:06
3,4,23404,Nutrient,FDBN00004,Fatty acids,0,NULL,low,NULL,NULL,...,1335,NULL,EAFUS,NULL,NULL,NULL,NULL,NULL,2014-11-05 13:46:00,2014-11-05 13:46:00
4,5,11134,Nutrient,FDBN00005,Fiber (dietary),0,NULL,low,NULL,NULL,...,NULL,NULL,DUKE,NULL,NULL,NULL,NULL,NULL,2014-11-05 13:47:36,2014-11-05 13:47:36


In [60]:
for column in nutrients.columns:
    print(column, nutrients[nutrients[column] == 'NULL'].shape[0])

id 0
legacy_id 0
type 0
public_id 0
name 0
export 0
state 38
annotation_quality 0
description 37
wikipedia_id 37
comments 38
dfc_id 38
duke_id 33
eafus_id 37
dfc_name 38
compound_source 0
metabolism 38
synthesis_citations 38
general_citations 38
creator_id 38
updater_id 38
created_at 0
updated_at 0


### Drop whole table?

# Exploratory data analysis
## Pathways table

In [68]:
pathways = pd.read_sql_query('''SELECT pa.* 
                            FROM pathways pa
                            ;''', engine)

2019-03-15 13:36:23,979 INFO sqlalchemy.engine.base.Engine SELECT pa.* 
                            FROM pathways pa
                            ;


INFO:sqlalchemy.engine.base.Engine:SELECT pa.* 
                            FROM pathways pa
                            ;


2019-03-15 13:36:23,979 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [69]:
pathways.head(20)

,id,smpdb_id,kegg_map_id,name,created_at,updated_at
0,1,SMP00006,map00350,Tyrosine Metabolism,2015-02-23 22:40:48,2015-02-23 22:40:48
1,2,SMP00068,map00150,Androgen and Estrogen Metabolism,2015-02-23 22:40:58,2015-02-23 22:40:58
2,3,SMP00011,map00562,Inositol Metabolism,2015-02-23 22:41:14,2015-02-23 22:41:14
3,4,SMP00462,map00562,Inositol Phosphate Metabolism,2015-02-23 22:41:14,2015-02-23 22:41:14
4,5,SMP00012,map00350,Catecholamine Biosynthesis,2015-02-23 22:42:38,2015-02-23 22:42:38
5,6,SMP00008,map00360,Phenylalanine and Tyrosine Metabolism,2015-02-23 22:42:38,2015-02-23 22:42:38
6,7,SMP00019,,Transcription/Translation,2015-02-23 22:42:38,2015-02-23 22:42:38
7,8,SMP00028,map00232,Caffeine Metabolism,2015-02-23 22:42:49,2015-02-23 22:42:49
8,9,SMP00037,map00310,Lysine Degradation,2015-02-23 22:42:58,2015-02-23 22:42:58
9,10,SMP00016,map00640,Propanoate Metabolism,2015-02-23 22:43:04,2015-02-23 22:43:04


### Drop created_at, updated_at

# Exploratory data analysis
## References table

In [79]:
references = pd.read_sql_query('''SELECT r.* 
                            FROM [references] r
                            ;''', engine)

2019-03-15 13:42:27,331 INFO sqlalchemy.engine.base.Engine SELECT r.* 
                            FROM [references] r
                            ;


INFO:sqlalchemy.engine.base.Engine:SELECT r.* 
                            FROM [references] r
                            ;


2019-03-15 13:42:27,333 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [78]:
table_names

['compound_alternate_parents',
 'compound_external_descriptors',
 'compound_substituents',
 'compound_synonyms',
 'compounds',
 'compounds_enzymes',
 'compounds_flavors',
 'compounds_health_effects',
 'compounds_pathways',
 'enzymes',
 'flavors',
 'food_taxonomies',
 'foodcomex_compound_providers',
 'foodcomex_compounds',
 'foods',
 'foods_copy',
 'health_effects',
 'nutrients',
 'pathways',
 'references']

In [80]:
references.head()

,id,ref_type,text,pubmed_id,link,title,creator_id,updater_id,created_at,updated_at,source_id,source_type
0,1,general,"Yannai, Shmuel. (2004) Dictionary of food comp...",,,,NULL,NULL,2015-02-23 22:39:47,2015-02-23 22:39:47,1,Compound
1,2,general,"Neveu V, Perez-Jimenez J, Vos F, Crespy V, du ...",,,,NULL,NULL,2015-02-23 22:39:50,2015-02-23 22:39:50,22,Compound
2,3,general,"Neveu V, Perez-Jimenez J, Vos F, Crespy V, du ...",,,,NULL,NULL,2015-02-23 22:39:50,2015-02-23 22:39:50,48,Compound
3,4,general,"de Villiers A, Vanhoenacker G, Majek P, Sandra...",,,,NULL,NULL,2015-02-23 22:39:52,2015-02-23 22:39:52,52,Compound
4,5,general,"Neveu V, Perez-Jimenez J, Vos F, Crespy V, du ...",,,,NULL,NULL,2015-02-23 22:39:54,2015-02-23 22:39:54,59,Compound


In [81]:
references.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31791 entries, 0 to 31790
Data columns (total 12 columns):
id             31791 non-null object
ref_type       31791 non-null object
text           31791 non-null object
pubmed_id      31791 non-null object
link           31791 non-null object
title          31791 non-null object
creator_id     31791 non-null object
updater_id     31791 non-null object
created_at     31791 non-null object
updated_at     31791 non-null object
source_id      31791 non-null object
source_type    31791 non-null object
dtypes: object(12)
memory usage: 2.9+ MB


### Drop table.

# Exploratory data analysis
## food taxonomies table

In [82]:
food_taxonomies = pd.read_sql_query('''SELECT ft.* 
                            FROM food_taxonomies ft
                            ;''', engine)

2019-03-15 13:45:43,423 INFO sqlalchemy.engine.base.Engine SELECT ft.* 
                            FROM food_taxonomies ft
                            ;


INFO:sqlalchemy.engine.base.Engine:SELECT ft.* 
                            FROM food_taxonomies ft
                            ;


2019-03-15 13:45:43,425 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [83]:
food_taxonomies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 7 columns):
id                      919 non-null object
food_id                 919 non-null object
ncbi_taxonomy_id        919 non-null object
classification_name     919 non-null object
classification_order    919 non-null object
created_at              919 non-null object
updated_at              919 non-null object
dtypes: object(7)
memory usage: 50.3+ KB


In [87]:
food_taxonomies.head()

,id,food_id,ncbi_taxonomy_id,classification_name,classification_order,created_at,updated_at
0,1,1,357850,\Eukaryota\,1,2017-03-29 18:35:53,2017-03-29 18:35:53
1,2,1,357850,\Viridiplantae\,2,2017-03-29 18:35:53,2017-03-29 18:35:53
2,3,1,357850,\Streptophyta\,3,2017-03-29 18:35:53,2017-03-29 18:35:53
3,4,1,357850,\Embryophyta\,4,2017-03-29 18:35:53,2017-03-29 18:35:53
4,5,1,357850,\Tracheophyta\,5,2017-03-29 18:35:53,2017-03-29 18:35:53


In [89]:
food_taxonomies['classification_order'].value_counts()

1     64
2     64
5     63
3     63
4     63
7     60
6     60
8     60
9     59
10    58
11    57
13    54
12    54
14    47
15    40
16    30
17    17
18     6
Name: classification_order, dtype: int64

In [93]:
food_taxonomies[food_taxonomies['classification_order'] == '16']

,id,food_id,ncbi_taxonomy_id,classification_name,classification_order,created_at,updated_at
15,16,1,357850,\apioid superclade\,16,2017-03-29 18:35:53,2017-03-29 18:35:53
33,34,19,72341,\Anthemideae\,16,2017-03-29 18:35:54,2017-03-29 18:35:54
51,52,37,3821,\Phaseoleae\,16,2017-03-29 18:35:54,2017-03-29 18:35:54
68,69,54,2708,\Citrus\,16,2017-03-29 18:35:54,2017-03-29 18:35:54
84,85,70,66014,\Cymbopogon\,16,2017-03-29 18:35:54,2017-03-29 18:35:54
100,101,86,4232,\Heliantheae alliance\,16,2017-03-29 18:35:55,2017-03-29 18:35:55
118,119,104,3869,\Genisteae\,16,2017-03-29 18:35:55,2017-03-29 18:35:55
135,136,121,4146,\Olea\,16,2017-03-29 18:35:55,2017-03-29 18:35:55
151,152,137,271192,\apioid superclade\,16,2017-03-29 18:35:56,2017-03-29 18:35:56
182,183,168,49988,\Mentheae\,16,2017-03-29 18:35:56,2017-03-29 18:35:56


In [95]:
food_taxonomies['ncbi_taxonomy_id'].value_counts()

271192    18
72341     18
941588    18
153159    18
4232      18
357850    18
227725    17
3869      17
49988     17
4072      17
87088     17
113210    17
225387    17
59499     17
40321     17
80720     17
3821      17
66014     16
8267      16
4146      16
172266    16
61886     16
108881    16
8101      16
3705      16
176259    16
138011    16
3728      16
183260    16
2708      16
          ..
151071    15
9940      15
29965     15
8113      15
271940    15
292385    15
59837     14
348707    14
8033      14
385388    14
9749      14
3603      14
8469      14
8014      13
175226    13
4682      13
78511     13
3564      13
8252      13
145700    13
77113     11
224910    11
49314     11
5341      10
6452       9
5192       8
7713       5
6448       5
6544       5
2763       2
Name: ncbi_taxonomy_id, Length: 64, dtype: int64

In [96]:
food_taxonomies[food_taxonomies['ncbi_taxonomy_id'] == '5192']

,id,food_id,ncbi_taxonomy_id,classification_name,classification_order,created_at,updated_at
681,682,756,5192,\Eukaryota\,1,2017-03-29 18:36:03,2017-03-29 18:36:03
682,683,756,5192,\Fungi\,2,2017-03-29 18:36:03,2017-03-29 18:36:03
683,684,756,5192,\Dikarya\,3,2017-03-29 18:36:03,2017-03-29 18:36:03
684,685,756,5192,\Ascomycota\,4,2017-03-29 18:36:03,2017-03-29 18:36:03
685,686,756,5192,\Pezizomycotina\,5,2017-03-29 18:36:03,2017-03-29 18:36:03
686,687,756,5192,\Pezizomycetes\,6,2017-03-29 18:36:03,2017-03-29 18:36:03
687,688,756,5192,\Pezizales\,7,2017-03-29 18:36:03,2017-03-29 18:36:03
688,689,756,5192,\Morchellaceae\,8,2017-03-29 18:36:03,2017-03-29 18:36:03


In [99]:
food_taxonomies['food_id'].value_counts()

19     18
137    18
249    18
849    18
86     18
1      18
168    17
342    17
104    17
466    17
201    17
436    17
37     17
909    17
814    17
403    17
879    17
887    16
797    16
527    16
599    16
54     16
121    16
786    16
70     16
799    16
185    16
420    16
218    16
748    16
       ..
388    15
483    15
359    15
316    15
723    15
622    15
513    14
801    14
726    14
374    14
302    14
571    14
585    14
267    13
453    13
880    13
289    13
155    13
548    13
798    13
802    11
331    11
863    11
561    10
280     9
756     8
617     5
861     5
544     5
615     2
Name: food_id, Length: 64, dtype: int64

### Disappointed to discover that the full taxonomy is apparently only available for 64 foods in the foods table.

### Drop id, created_at, updated_at. 

Keep food_id INTEGER foreign key, ncbi_taxonomy_id INTEGER, classification_name TEXT, classification_order INTEGER

<a id = 'test_query_ft'></a>

### Test query for food taxonomies to see how they'll look

In [119]:
pd.read_sql_query('''SELECT f.name, f.name_scientific,
                            ft.classification_name, 
                            ft.classification_order
                        FROM food_taxonomies ft
                        JOIN foods f ON f.id == ft.food_id
                        WHERE ft.food_id == 756

                        ORDER BY ft.classification_order
                    ;''', engine)

2019-03-15 14:18:28,726 INFO sqlalchemy.engine.base.Engine SELECT f.name, f.name_scientific,
                            ft.classification_name, 
                            ft.classification_order
                        FROM food_taxonomies ft
                        JOIN foods f ON f.id == ft.food_id
                        WHERE ft.food_id == 756

                        ORDER BY ft.classification_order
                    ;


INFO:sqlalchemy.engine.base.Engine:SELECT f.name, f.name_scientific,
                            ft.classification_name, 
                            ft.classification_order
                        FROM food_taxonomies ft
                        JOIN foods f ON f.id == ft.food_id
                        WHERE ft.food_id == 756

                        ORDER BY ft.classification_order
                    ;


2019-03-15 14:18:28,736 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,name,name_scientific,classification_name,classification_order
0,Morchella (Morel),Morchellaceae,\Eukaryota\,1
1,Morchella (Morel),Morchellaceae,\Fungi\,2
2,Morchella (Morel),Morchellaceae,\Dikarya\,3
3,Morchella (Morel),Morchellaceae,\Ascomycota\,4
4,Morchella (Morel),Morchellaceae,\Pezizomycotina\,5
5,Morchella (Morel),Morchellaceae,\Pezizomycetes\,6
6,Morchella (Morel),Morchellaceae,\Pezizales\,7
7,Morchella (Morel),Morchellaceae,\Morchellaceae\,8


# Exploratory data analysis
## foodcomex compound providers  table

In [ ]:
foodcomex_compound_providers = pd.read_sql_query('''SELECT fcp.* 
                            FROM foodcomex_compound_providers fcp
                            ;''', engine)

# Exploratory data analysis
## foodcomex compounds  table

In [ ]:
foodcomex_compounds = pd.read_sql_query('''SELECT fc.* 
                            FROM foodcomex_compounds fc
                            ;''', engine)

# Exploratory data analysis
## compounds table

In [ ]:
compounds = pd.read_sql_query('''SELECT c.* 
                            FROM compounds c
                            ;''', engine)

# Exploratory data analysis
## compound alternate parents table

In [ ]:
compound_alternate_parents = pd.read_sql_query('''SELECT cap.* 
                            FROM compounds_alternate_parents cap
                            ;''', engine)

# Exploratory data analysis
## compound external descriptors

In [ ]:
compound_external_descriptors = pd.read_sql_query('''SELECT ced.* 
                            FROM compound_external_descriptors ced
                            ;''', engine)

# Exploratory data analysis
## compound substituents

In [ ]:
compound_substituents = pd.read_sql_query('''SELECT csu.* 
                            FROM compound_substituents csu
                            ;''', engine)

# Exploratory data analysis
## compound synonyms table

In [ ]:
compound_synonyms = pd.read_sql_query('''SELECT csy.* 
                            FROM compound_synonyms csy
                            ;''', engine)

In [ ]:
['compound_alternate_parents',
 'compound_external_descriptors',
 'compound_substituents',
 'compound_synonyms',
 'compounds',
 'compounds_enzymes',
 'compounds_flavors',
 'compounds_health_effects',
 'compounds_pathways',
 'enzymes',
 'flavors',
 'food_taxonomies',
 'foodcomex_compound_providers',
 'foodcomex_compounds',
 'foods',
 'health_effects',
 'nutrients',
 'pathways',
 'references']

# Exploratory data analysis
## compounds enzymes table

In [ ]:
compounds_enzymes = pd.read_sql_query('''SELECT ce.* 
                            FROM compounds_enzymes ce
                            ;''', engine)

# Exploratory data analysis
## compounds flavors table

In [ ]:
compounds_flavors = pd.read_sql_query('''SELECT cf.* 
                            FROM compounds_flavors cf
                            ;''', engine)

# Exploratory data analysis
## compounds health effects table

In [ ]:
compounds_health_effects = pd.read_sql_query('''SELECT che.* 
                            FROM compounds_health_effects che
                            ;''', engine)

# Exploratory data analysis
## compounds pathways table

In [ ]:
compounds_pathways = pd.read_sql_query('''SELECT cp.* 
                            FROM compounds_pathways cp
                            ;''', engine)

<a id = 'droppingcolumns'></a>

[(Back to top)](#top)

# Dropping columns
## Dropping columns from food table

In [ ]:
#### EXAMPLE

BEGIN TRANSACTION;
 
ALTER TABLE table RENAME TO temp_table;
 
CREATE TABLE table
( 
   column_definition,
   ...
);
 
INSERT INTO table (column_list)
  SELECT column_list
  FROM temp_table;
 
DROP TABLE temp_table;
 
COMMIT;

In [5]:
# connect to db
connection = sqlite3.connect('foodb.db')

In [6]:
cur = connection.cursor()

In [23]:
#Look at columns in foods table
cur.execute('''PRAGMA table_info(foods)
;''',).fetchall()

[(0, 'id', '', 0, None, 0),
 (1, 'name', '', 0, None, 0),
 (2, 'name_scientific', '', 0, None, 0),
 (3, 'description', '', 0, None, 0),
 (4, 'itis_id', '', 0, None, 0),
 (5, 'wikipedia_id', '', 0, None, 0),
 (6, 'wikipedia_id_img', '', 0, None, 0),
 (7, 'picture_content_type', '', 0, None, 0),
 (8, 'picture_file_size', '', 0, None, 0),
 (9, 'picture_updated_at', '', 0, None, 0),
 (10, 'legacy_id', '', 0, None, 0),
 (11, 'food_group', '', 0, None, 0),
 (12, 'food_subgroup', '', 0, None, 0),
 (13, 'food_type', '', 0, None, 0),
 (14, 'created_at', '', 0, None, 0),
 (15, 'updated_at', '', 0, None, 0),
 (16, 'creator_id', '', 0, None, 0),
 (17, 'updater_id', '', 0, None, 0),
 (18, 'export_to_afcdb', '', 0, None, 0),
 (19, 'category', '', 0, None, 0),
 (20, 'ncbi_taxonomy_id', '', 0, None, 0),
 (21, 'export_to_foodb', '', 0, None, 0)]

In [12]:
#Create a new foods table
cur.execute('''CREATE TABLE foods_copy
                (id INTEGER PRIMARY KEY, name TEXT, name_scientific TEXT, 
                description TEXT, wikipedia_id TEXT, wikipedia_id_img TEXT,
                food_group TEXT, food_subgroup TEXT, food_type TEXT, 
                category TEXT, ncbi_taxonomy_id INTEGER)
            ;''')

In [14]:
#Insert values into columns of new table
cur.execute('''INSERT INTO foods_copy
                    (id, name, name_scientific, 
                    description, wikipedia_id, wikipedia_id_img,
                    food_group, food_subgroup, food_type, 
                    category, ncbi_taxonomy_id)
                SELECT id, name, name_scientific, 
                    description, wikipedia_id, wikipedia_id_img, 
                    food_group, food_subgroup, food_type, 
                    category, ncbi_taxonomy_id
                FROM foods
            ;''')

In [15]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM foods_copy
                LIMIT 5
            ;''').fetchall()

#Check that columns were populated from old table
#Doesn't work before commit. Works after commit.
#pd.read_sql_query('''SELECT * 
#                        FROM foods_copy
#                    ;''', engine)

[(1,
  'Angelica',
  'Angelica keiskei',
  'Angelica is a genus of about 60 species of tall biennial and perennial herbs in the family Apiaceae, native to temperate and subarctic regions of the Northern Hemisphere, reaching as far north as Iceland and Lapland. They grow to 1äóñ3 m tall, with large bipinnate leaves and large compound umbels of white or greenish-white flowers. Some species can be found in purple moor and rush pastures.',
  'Angelica',
  '1.jpg',
  'Herbs and Spices',
  'Herbs',
  'Type 1',
  'specific',
  357850),
 (2,
  'Savoy cabbage',
  'Brassica oleracea var. sabauda',
  'Savoy cabbage (Brassica oleracea convar. capitata var. sabauda L. ) is a variety of the cabbage, a cultivar of the plant species Brassica oleracea. Savoy cabbage is a winter vegetable. A variety of the savoy cabbage is the January King Cabbage. Savoy cabbage can be used in a variety of recipes. It pairs well with red wine, apples, spices, horseradish and meat. It can be used for roulades, in stews a

In [56]:
foods_copy_Nones = foods_copy['ncbi_taxonomy_id'] == ''

In [57]:
foods_copy_Nones.value_counts()

False    631
True     276
Name: ncbi_taxonomy_id, dtype: int64

Null values are not reading as Null but there are empty values.

In [16]:
#Check data types of new table
#Works before commit

cur.execute('''PRAGMA table_info(foods_copy)
;''',).fetchall()

#Check data types of new table
#Only works after commit
#New table dropped 90kb from memory compared to old table
#foods_copy = pd.read_sql_query('''SELECT * 
#                        FROM foods_copy
#                    ;''', engine)
#foods_copy.info()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'name_scientific', 'TEXT', 0, None, 0),
 (3, 'description', 'TEXT', 0, None, 0),
 (4, 'wikipedia_id', 'TEXT', 0, None, 0),
 (5, 'wikipedia_id_img', 'TEXT', 0, None, 0),
 (6, 'food_group', 'TEXT', 0, None, 0),
 (7, 'food_subgroup', 'TEXT', 0, None, 0),
 (8, 'food_type', 'TEXT', 0, None, 0),
 (9, 'category', 'TEXT', 0, None, 0),
 (10, 'ncbi_taxonomy_id', 'INTEGER', 0, None, 0)]

In [ ]:
#Check columns of new table
cur.execute('''PRAGMA table_info(foods_copy)
;''',).fetchall()

In [17]:
#Delete the old table
cur.execute('''DROP TABLE foods;''')

In [18]:
#Rename new table to  foods
cur.execute('''ALTER TABLE foods_copy
                RENAME TO foods
                ;''')

In [19]:
#Check list of tables in database
cur.execute('''SELECT name
                FROM sqlite_master 
                WHERE type ='table' 
                    AND name NOT LIKE 'sqlite_%'
            ;''').fetchall()

[('compound_alternate_parents',),
 ('compound_external_descriptors',),
 ('compound_substituents',),
 ('compound_synonyms',),
 ('compounds_enzymes',),
 ('compounds_flavors',),
 ('compounds_health_effects',),
 ('compounds_pathways',),
 ('compounds',),
 ('enzymes',),
 ('flavors',),
 ('food_taxonomies',),
 ('foodcomex_compound_providers',),
 ('foodcomex_compounds',),
 ('health_effects',),
 ('nutrients',),
 ('pathways',),
 ('references',),
 ('foods',)]

In [20]:
#Check columns of new table
cur.execute('''PRAGMA table_info(foods)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'name_scientific', 'TEXT', 0, None, 0),
 (3, 'description', 'TEXT', 0, None, 0),
 (4, 'wikipedia_id', 'TEXT', 0, None, 0),
 (5, 'wikipedia_id_img', 'TEXT', 0, None, 0),
 (6, 'food_group', 'TEXT', 0, None, 0),
 (7, 'food_subgroup', 'TEXT', 0, None, 0),
 (8, 'food_type', 'TEXT', 0, None, 0),
 (9, 'category', 'TEXT', 0, None, 0),
 (10, 'ncbi_taxonomy_id', 'INTEGER', 0, None, 0)]

In [21]:
# Save (commit) the changes
connection.commit()

In [59]:
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
connection.close()

[(Back to top)](#top)
## Dropping columns from enzymes table

In [25]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(enzymes)
;''',).fetchall()

[(0, 'id', '', 0, None, 0),
 (1, 'name', '', 0, None, 0),
 (2, 'gene_name', '', 0, None, 0),
 (3, 'description', '', 0, None, 0),
 (4, 'go_classification', '', 0, None, 0),
 (5, 'general_function', '', 0, None, 0),
 (6, 'specific_function', '', 0, None, 0),
 (7, 'pathway', '', 0, None, 0),
 (8, 'reaction', '', 0, None, 0),
 (9, 'cellular_location', '', 0, None, 0),
 (10, 'signals', '', 0, None, 0),
 (11, 'transmembrane_regions', '', 0, None, 0),
 (12, 'molecular_weight', '', 0, None, 0),
 (13, 'theoretical_pi', '', 0, None, 0),
 (14, 'locus', '', 0, None, 0),
 (15, 'chromosome', '', 0, None, 0),
 (16, 'uniprot_name', '', 0, None, 0),
 (17, 'uniprot_id', '', 0, None, 0),
 (18, 'pdb_id', '', 0, None, 0),
 (19, 'genbank_protein_id', '', 0, None, 0),
 (20, 'genbank_gene_id', '', 0, None, 0),
 (21, 'genecard_id', '', 0, None, 0),
 (22, 'genatlas_id', '', 0, None, 0),
 (23, 'hgnc_id', '', 0, None, 0),
 (24, 'hprd_id', '', 0, None, 0),
 (25, 'organism', '', 0, None, 0),
 (26, 'general_citatio

In [26]:
#Create a new enzymes table
cur.execute('''CREATE TABLE enzymes_copy
                (id INTEGER PRIMARY KEY, name TEXT, gene_name TEXT, 
                uniprot_id TEXT)
            ;''')

In [29]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(enzymes_copy)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'gene_name', 'TEXT', 0, None, 0),
 (3, 'uniprot_id', 'TEXT', 0, None, 0)]

In [31]:
#Insert values into columns of new table
cur.execute('''INSERT INTO enzymes_copy
                    (id, name, gene_name, uniprot_id)
                SELECT id, name, gene_name, uniprot_id
                FROM enzymes
            ;''')

In [32]:
#Check data types of new table
#Works before commit

cur.execute('''PRAGMA table_info(enzymes_copy)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'gene_name', 'TEXT', 0, None, 0),
 (3, 'uniprot_id', 'TEXT', 0, None, 0)]

In [34]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM enzymes_copy
                LIMIT 5
            ;''').fetchall()

[(1, 'UDP-glucuronosyltransferase 2B28', 'UGT2B28', 'Q9BY64'),
 (2, 'Estrogen receptor beta', 'ESR2', 'Q92731'),
 (3, 'UDP-glucuronosyltransferase 2B4', 'UGT2B4', 'P06133'),
 (4, 'UDP-glucuronosyltransferase 1-4', 'UGT1A4', 'P22310'),
 (5, 'UDP-glucuronosyltransferase 2B10', 'UGT2B10', 'P36537')]

In [35]:
#Delete the old table
cur.execute('''DROP TABLE enzymes;''')

In [36]:
#Rename new table to enzymes
cur.execute('''ALTER TABLE enzymes_copy
                RENAME TO enzymes
                ;''')

In [37]:
#Check list of tables in database
cur.execute('''SELECT name
                FROM sqlite_master 
                WHERE type ='table' 
                    AND name NOT LIKE 'sqlite_%'
            ;''').fetchall()

[('compound_alternate_parents',),
 ('compound_external_descriptors',),
 ('compound_substituents',),
 ('compound_synonyms',),
 ('compounds_enzymes',),
 ('compounds_flavors',),
 ('compounds_health_effects',),
 ('compounds_pathways',),
 ('compounds',),
 ('flavors',),
 ('food_taxonomies',),
 ('foodcomex_compound_providers',),
 ('foodcomex_compounds',),
 ('health_effects',),
 ('nutrients',),
 ('pathways',),
 ('references',),
 ('foods',),
 ('enzymes',)]

In [39]:
#Check columns of new table
cur.execute('''PRAGMA table_info(enzymes)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'gene_name', 'TEXT', 0, None, 0),
 (3, 'uniprot_id', 'TEXT', 0, None, 0)]

In [40]:
# Save (commit) the changes
connection.commit()

[(Back to top)](#top)
## Dropping columns from health_effects table

In [41]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(health_effects)
;''',).fetchall()

[(0, 'id', '', 0, None, 0),
 (1, 'name', '', 0, None, 0),
 (2, 'description', '', 0, None, 0),
 (3, 'chebi_name', '', 0, None, 0),
 (4, 'chebi_id', '', 0, None, 0),
 (5, 'created_at', '', 0, None, 0),
 (6, 'updated_at', '', 0, None, 0),
 (7, 'creator_id', '', 0, None, 0),
 (8, 'updater_id', '', 0, None, 0),
 (9, 'chebi_definition', '', 0, None, 0)]

In [42]:
#Create a new enzymes table
cur.execute('''CREATE TABLE health_effects_copy
                (id INTEGER PRIMARY KEY, name TEXT, description TEXT, 
                chebi_name TEXT, chebi_id INTEGER, chebi_definition TEXT)
            ;''')

#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(health_effects_copy)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'description', 'TEXT', 0, None, 0),
 (3, 'chebi_name', 'TEXT', 0, None, 0),
 (4, 'chebi_id', 'INTEGER', 0, None, 0),
 (5, 'chebi_definition', 'TEXT', 0, None, 0)]

In [43]:
#Insert values into columns of new table
cur.execute('''INSERT INTO health_effects_copy
                    (id, name, description,
                    chebi_name, chebi_id, chebi_definition)
                SELECT id, name, description,
                    chebi_name, chebi_id, chebi_definition
                FROM health_effects
            ;''')
#Check data types of new table
#Works before commit

cur.execute('''PRAGMA table_info(health_effects_copy)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'description', 'TEXT', 0, None, 0),
 (3, 'chebi_name', 'TEXT', 0, None, 0),
 (4, 'chebi_id', 'INTEGER', 0, None, 0),
 (5, 'chebi_definition', 'TEXT', 0, None, 0)]

In [44]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM health_effects_copy
                LIMIT 5
            ;''').fetchall()

[(1,
  '(+)-inotropic',
  'An agent that alters the force or energy of muscular contractions. Positively inotropic agents increase the strength of muscular contraction.',
  'NULL',
  'NULL',
  'NULL'),
 (2,
  '(-)-chronotropic',
  'An agent that may change theÂ\xa0heartÂ\xa0rate by affecting theÂ\xa0nervesÂ\xa0controlling the heart, or by changing theÂ\xa0rhythmÂ\xa0produced by theÂ\xa0sinoatrial node. Negative chronotropes decrease heart rate.',
  'NULL',
  'NULL',
  'NULL'),
 (3,
  '(-)-inotropic',
  'An agent that alters the force or energy of muscular contractions. NegativelyÂ\xa0inotropicÂ\xa0agents weaken the force of muscular contractions.',
  'NULL',
  'NULL',
  'NULL'),
 (4,
  '11beta-hydroxysteroid-dehydrogenase inhibitor',
  'NULL',
  'enzyme inhibitor',
  23924,
  'A compound or agent that combines with an enzyme in such a manner as to prevent the normal substrate-enzyme combination and the catalytic reaction.'),
 (5,
  '12-lipoxygenase inhibitor',
  'NULL',
  'enzyme inhib

In [45]:
#Delete the old table
cur.execute('''DROP TABLE health_effects;''')

#Rename new table to enzymes
cur.execute('''ALTER TABLE health_effects_copy
                RENAME TO health_effects
                ;''')
#Check list of tables in database
cur.execute('''SELECT name
                FROM sqlite_master 
                WHERE type ='table' 
                    AND name NOT LIKE 'sqlite_%'
            ;''').fetchall()

[('compound_alternate_parents',),
 ('compound_external_descriptors',),
 ('compound_substituents',),
 ('compound_synonyms',),
 ('compounds_enzymes',),
 ('compounds_flavors',),
 ('compounds_health_effects',),
 ('compounds_pathways',),
 ('compounds',),
 ('flavors',),
 ('food_taxonomies',),
 ('foodcomex_compound_providers',),
 ('foodcomex_compounds',),
 ('nutrients',),
 ('pathways',),
 ('references',),
 ('foods',),
 ('enzymes',),
 ('health_effects',)]

In [46]:
#Check columns of new table
cur.execute('''PRAGMA table_info(health_effects)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'description', 'TEXT', 0, None, 0),
 (3, 'chebi_name', 'TEXT', 0, None, 0),
 (4, 'chebi_id', 'INTEGER', 0, None, 0),
 (5, 'chebi_definition', 'TEXT', 0, None, 0)]

In [47]:
# Save (commit) the changes
connection.commit()

[(Back to top)](#top)
## Dropping columns from flavors table

In [48]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(flavors)
;''',).fetchall()

[(0, 'id', '', 0, None, 0),
 (1, 'name', '', 0, None, 0),
 (2, 'flavor_group', '', 0, None, 0),
 (3, 'category', '', 0, None, 0),
 (4, 'created_at', '', 0, None, 0),
 (5, 'updated_at', '', 0, None, 0),
 (6, 'creator_id', '', 0, None, 0),
 (7, 'updater_id', '', 0, None, 0)]

In [49]:
#Create a new enzymes table
cur.execute('''CREATE TABLE flavors_copy
                (id INTEGER PRIMARY KEY, name TEXT, flavor_group TEXT)
            ;''')

#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(flavors_copy)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'flavor_group', 'TEXT', 0, None, 0)]

In [50]:
#Insert values into columns of new table
cur.execute('''INSERT INTO flavors_copy
                    (id, name, flavor_group)
                SELECT id, name, flavor_group
                FROM flavors
            ;''')
#Check data types of new table
#Works before commit

cur.execute('''PRAGMA table_info(flavors_copy)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'flavor_group', 'TEXT', 0, None, 0)]

In [51]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM flavors_copy
                LIMIT 5
            ;''').fetchall()

[(1, 'celery', 'vegetable'),
 (2, 'corn', 'vegetable'),
 (3, 'cucumber', 'vegetable'),
 (4, 'horseradish', 'vegetable'),
 (5, 'vegetable', 'vegetable')]

In [52]:
#Delete the old table
cur.execute('''DROP TABLE flavors;''')

#Rename new table to enzymes
cur.execute('''ALTER TABLE flavors_copy
                RENAME TO flavors
                ;''')
#Check list of tables in database
cur.execute('''SELECT name
                FROM sqlite_master 
                WHERE type ='table' 
                    AND name NOT LIKE 'sqlite_%'
            ;''').fetchall()

[('compound_alternate_parents',),
 ('compound_external_descriptors',),
 ('compound_substituents',),
 ('compound_synonyms',),
 ('compounds_enzymes',),
 ('compounds_flavors',),
 ('compounds_health_effects',),
 ('compounds_pathways',),
 ('compounds',),
 ('food_taxonomies',),
 ('foodcomex_compound_providers',),
 ('foodcomex_compounds',),
 ('nutrients',),
 ('pathways',),
 ('references',),
 ('foods',),
 ('enzymes',),
 ('health_effects',),
 ('flavors',)]

In [53]:
#Check columns of new table
cur.execute('''PRAGMA table_info(flavors)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'flavor_group', 'TEXT', 0, None, 0)]

In [54]:
# Save (commit) the changes
connection.commit()

[(Back to top)](#top)
## Dropping columns from pathways table

In [72]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(pathways)
;''',).fetchall()

[(0, 'id', '', 0, None, 0),
 (1, 'smpdb_id', '', 0, None, 0),
 (2, 'kegg_map_id', '', 0, None, 0),
 (3, 'name', '', 0, None, 0),
 (4, 'created_at', '', 0, None, 0),
 (5, 'updated_at', '', 0, None, 0)]

In [73]:
#Create a new  table
cur.execute('''CREATE TABLE pathways_copy
                (id INTEGER PRIMARY KEY, smpdb_id TEXT, kegg_map_id TEXT, 
                name TEXT)
            ;''')

#Insert values into columns of new table
cur.execute('''INSERT INTO pathways_copy
                    (id, smpdb_id, kegg_map_id, name)
                SELECT id, smpdb_id, kegg_map_id, name
                FROM pathways
            ;''')

#Delete the old table
cur.execute('''DROP TABLE pathways;''')

#Rename new table to enzymes
cur.execute('''ALTER TABLE pathways_copy
                RENAME TO pathways
                ;''')

In [74]:
#Check columns of new table
cur.execute('''PRAGMA table_info(pathways)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'smpdb_id', 'TEXT', 0, None, 0),
 (2, 'kegg_map_id', 'TEXT', 0, None, 0),
 (3, 'name', 'TEXT', 0, None, 0)]

In [75]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM pathways
                LIMIT 3
            ;''').fetchall()

[(1, 'SMP00006', 'map00350', 'Tyrosine Metabolism'),
 (2, 'SMP00068', 'map00150', 'Androgen and Estrogen Metabolism'),
 (3, 'SMP00011', 'map00562', 'Inositol Metabolism')]

In [76]:
# Save (commit) the changes
connection.commit()

[(Back to top)](#top)
## Dropping columns from food_taxonomies table

In [100]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info(food_taxonomies)
;''',).fetchall()

[(0, 'id', '', 0, None, 0),
 (1, 'food_id', '', 0, None, 0),
 (2, 'ncbi_taxonomy_id', '', 0, None, 0),
 (3, 'classification_name', '', 0, None, 0),
 (4, 'classification_order', '', 0, None, 0),
 (5, 'created_at', '', 0, None, 0),
 (6, 'updated_at', '', 0, None, 0)]

In [105]:
#Create a new  table
cur.execute('''CREATE TABLE food_taxonomies_copy
                (id INTEGER, food_id INTEGER, 
                ncbi_taxonomy_id INTEGER, classification_name TEXT, 
                classification_order INTEGER,
                FOREIGN KEY(food_id) REFERENCES foods(id))
            ;''')

In [107]:
#Insert values into columns of new table
cur.execute('''INSERT INTO food_taxonomies_copy
                    (id, food_id, ncbi_taxonomy_id, classification_name,
                    classification_order)
                SELECT id, food_id, ncbi_taxonomy_id, classification_name,
                    classification_order
                FROM food_taxonomies
            ;''')

#Delete the old table
cur.execute('''DROP TABLE food_taxonomies;''')

#Rename new table to enzymes
cur.execute('''ALTER TABLE food_taxonomies_copy
                RENAME TO food_taxonomies
                ;''')

In [108]:
#Check columns of new table
cur.execute('''PRAGMA table_info(food_taxonomies)
;''',).fetchall()

[(0, 'id', 'INTEGER', 0, None, 0),
 (1, 'food_id', 'INTEGER', 0, None, 0),
 (2, 'ncbi_taxonomy_id', 'INTEGER', 0, None, 0),
 (3, 'classification_name', 'TEXT', 0, None, 0),
 (4, 'classification_order', 'INTEGER', 0, None, 0)]

In [109]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM food_taxonomies
                LIMIT 3
            ;''').fetchall()

[(1, 1, 357850, '\\Eukaryota\\', 1),
 (2, 1, 357850, '\\Viridiplantae\\', 2),
 (3, 1, 357850, '\\Streptophyta\\', 3)]

In [110]:
# Save (commit) the changes
connection.commit()

[(Back to top)](#top)
## Dropping columns from [    ] table

In [ ]:
#Look at columns in enzymes table
cur.execute('''PRAGMA table_info()
;''',).fetchall()

In [ ]:
#Create a new  table
cur.execute('''CREATE TABLE e_copy
                (id INTEGER PRIMARY KEY, name TEXT, gene_name TEXT, 
                uniprot_id TEXT)
            ;''')

#Look at columns in  table
cur.execute('''PRAGMA table_info(_copy)
;''',).fetchall()

In [ ]:
#Insert values into columns of new table
cur.execute('''INSERT INTO _copy
                    (id, name, gene_name, uniprot_id)
                SELECT id, name, gene_name, uniprot_id
                FROM 
            ;''')
#Check data types of new table
#Works before commit

cur.execute('''PRAGMA table_info(_copy)
;''',).fetchall()

In [ ]:
#Check that columns were populated from old table
#This function works without committing 
cur.execute('''SELECT *
                FROM _copy
                LIMIT 5
            ;''').fetchall()

In [ ]:
#Delete the old table
cur.execute('''DROP TABLE ;''')

#Rename new table to enzymes
cur.execute('''ALTER TABLE _copy
                RENAME TO 
                ;''')
#Check list of tables in database
cur.execute('''SELECT name
                FROM sqlite_master 
                WHERE type ='table' 
                    AND name NOT LIKE 'sqlite_%'
            ;''').fetchall()

In [ ]:
#Check columns of new table
cur.execute('''PRAGMA table_info()
;''',).fetchall()

In [ ]:
# Save (commit) the changes
connection.commit()

<a id = 'dropping_tables'></a>

[(Back to top)](#top)
# Dropping whole tables

In [122]:
cur.execute('''DROP TABLE nutrients;''')

OperationalError: no such table: nutrients

In [123]:
cur.execute('''DROP TABLE [references];''')

In [124]:
# Save (commit) the changes
connection.commit()

In [125]:
connection.close()

In [ ]:
###############################################################################################

In [ ]:
#This encoding works with Windows.
foods = pd.read_csv('foods.csv', encoding = "ISO-8859-1")




In [ ]:
foods.head()

In [ ]:
compounds = pd.read_csv('compounds.csv', encoding = "ISO-8859-1")

In [ ]:
compounds_flavors = pd.read_csv('compounds_flavors.csv', encoding = "ISO-8859-1")

In [ ]:
foodcomex_compounds = pd.read_csv('foodcomex_compounds.csv', encoding = "ISO-8859-1")